In [1]:
import requests
api_key = ""
#WARNING: SET YOUR API KEY HERE more info: https://developer.riotgames.com
#WARNING: DO NOT PUSH THIS KEY TO GITHUB OR ANYWHERE ELSE PUBLIC (by Samynhn)

In [2]:
# get summoner info
summoner_name = "God Thunder"
summoner_name_for_save_path = "Zeus"
region = "asia"
contry = "KR"
SUMMONER_V4 = "https://" + contry + ".api.riotgames.com/lol/summoner/v4/summoners/by-name/"
SUMMONER_V4_URL = SUMMONER_V4 + summoner_name + "?api_key=" + api_key
response = requests.get(SUMMONER_V4_URL)
summoner_data = response.json()
encryptedPUUID = summoner_data['puuid']
print(encryptedPUUID)
# it may like : bEN5d9HH5CGcQR4N9M-s5fEfufdFMSgNUyQLam8cmtxy8krWZQe22c5TVuQKe23yb6hg9xsT6gKmXw

2WPMYAxqvcBIa0VTCZeeOD4-rNjZuPj2wmiSxfReP6698MOsbzuN_YK482OSaywApucVPNig0GKIgw


In [3]:

# get game detail
import json
def get_game_detail(game_name, region="asia", api_key=api_key):
    MATCH_V5_1 = "https://" +region+ ".api.riotgames.com/lol/match/v5/matches/"
    MATCH_V4_1_URL = MATCH_V5_1 + game_name + "?api_key=" + api_key
    response = requests.get(MATCH_V4_1_URL)
    data = response.json()
    return data
game_detail = get_game_detail("KR_6796022666")
json_response = json.dumps(game_detail) # aviod single quote in json
print(json_response) 
#it may like: {"metadata": {"dataVersion": "2", "matchId": "KR_6796022666", "participants": ["ZsTHjo...

{"metadata": {"dataVersion": "2", "matchId": "KR_6796022666", "participants": ["ZsTHjonxAsBQlaAd5FV3yCY0qEUZyGhrAIKs4mDJKb4BDaqG27P_6jlKVYssYxXdbZjywwi1jLyMrA", "18Ttib-yhRcIxwwgFqZpnAZvJoDnyjU40tme_u4m6eNhdTutS1V2h0_i1QS2UEY_c1LVnmxaiWpk0Q", "bEN5d9HH5CGcQR4N9M-s5fEfufdFMSgNUyQLam8cmtxy8krWZQe22c5TVuQKe23yb6hg9xsT6gKmXw", "1BebjTGXdc4N09AWL9EJFX5hyyVZu8T75tDxAuBKLeW7_jKGRVmWP5f42Lszhri1CjnoWdC73sT9jg", "tFBy-mqJEgC2NCo4OZEQhtqOn21Q5fi377J-4v_RS68VnffY98R9R8y2n_cJjwSvUmdxfzKdN13P_A", "rPGt3i0WgIZqx-9A9KDz2egqWxngL7lk4uTaP9d7J_r-lZvejWGwVzgiom43ZzDnqaUI8MexRPt0gA", "x58IPmmgdF2CkseIWK6HpX9fCdR6olmDZZEdRMoVJqWXym-bw26OTd4R9KROpa5hRoOhlqZxjPv-bQ", "12vWUD0mIVKdhsj6NdoFpkIls5bHDWuMl3qE6r7mFTnQdqsGm6bqLSGiaQqo_V0HJ_zJdsxwrKw1Cg", "7qVxqjNAhcWkh-qswUTFi9tJR5hBXSLGNYtn7tHdflygVVFLqnuxNVmnmOUTh_wuz8Ah8TMprBMRQw", "57Qe8PjGpga7nfiyqkzLLG-ltCq_dJLm7jUFeIZ4DBYJTwi2RhGK4PSoe8kUDaXZ3AydpCCidzA4Hw"]}, "info": {"gameCreation": 1699885652548, "gameDuration": 921, "gameEndTimestamp": 1699886633215, "ga

In [5]:
#output format: [summoner_name, summoner_team, summoner_position, summoner_champion, summoner_winFlag, opponent_name, opponent_team, opponent_position, opponent_champion, opponent_winFlag, gameMode, gameType]
def get_summoner_and_opponent_info(data, encryptedPUUID=encryptedPUUID):
    gameMode = data['info']['gameMode']
    # print(gameMode)
    gameType = data['info']['gameType']
    # print(gameType)
    gameVersion = data['info']['gameVersion']
    
    champion_names = [participant['championName'] for participant in data['info']['participants']]
    teamPositions = [participant['teamPosition'] for participant in data['info']['participants']]
    summonerNames = [participant['summonerName'] for participant in data['info']['participants']]
    winFlag = [participant['win'] for participant in data['info']['participants']]
    teams = ["red", "red", "red", "red", "red", "blue", "blue", "blue", "blue", "blue"]

    #get summoner's position, champion
    participants = data['metadata']['participants']
    summoner_index = participants.index(encryptedPUUID)
    summoner_name = summonerNames[summoner_index]
    summoner_team = teams[summoner_index]
    summoner_position = teamPositions[summoner_index]
    summoner_champion = champion_names[summoner_index]
    summoner_winFlag = winFlag[summoner_index]

    # get summoner's opponent position, champion
    def find_opponent_index(my_index, team_size=5):
        # 如果是红队，加上团队大小；如果是蓝队，减去团队大小
        opponent_index = (my_index + team_size) % (2 * team_size)
        return opponent_index
    opponent_index = find_opponent_index(summoner_index)
    opponent_name = summonerNames[opponent_index]
    opponent_team = teams[opponent_index]
    opponent_position = teamPositions[opponent_index]
    opponent_champion = champion_names[opponent_index]
    opponent_winFlag = winFlag[opponent_index]

    return [gameVersion, summoner_name, summoner_team, summoner_position, summoner_champion, summoner_winFlag, opponent_name, opponent_team, opponent_position, opponent_champion, opponent_winFlag, gameMode, gameType]


In [6]:
import csv
import os
def summoner_and_opponent_info_writer(output_path, summoner_and_opponent_info):
    fieldnames = ["gameVersion", "summoner_name", "summoner_team", "summoner_position", "summoner_champion", "summoner_winFlag", "opponent_name", "opponent_team", "opponent_position", "opponent_champion", "opponent_winFlag"]
    
    # 检查文件是否已存在，如果不存在则写入表头
    file_exists = os.path.isfile(output_path)

    with open(output_path, 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        # 仅在文件不存在时写入表头
        if not file_exists:
            writer.writeheader()
        
        writer.writerow({
            "gameVersion": summoner_and_opponent_info[0],
            "summoner_name": summoner_and_opponent_info[1],
            "summoner_team": summoner_and_opponent_info[2],
            "summoner_position": summoner_and_opponent_info[3],
            "summoner_champion": summoner_and_opponent_info[4],
            "summoner_winFlag": summoner_and_opponent_info[5],
            "opponent_name": summoner_and_opponent_info[6],
            "opponent_team": summoner_and_opponent_info[7],
            "opponent_position": summoner_and_opponent_info[8],
            "opponent_champion": summoner_and_opponent_info[9],
            "opponent_winFlag": summoner_and_opponent_info[10]
        })

In [7]:
#save game_name_arr to json file
import json
def save_game_name_arr_to_json(output_path, game_name_arr):
    with open(output_path, 'w') as outfile:
        json.dump(game_name_arr, outfile)



In [10]:
# According to the Riot API documentation, the rate limit for a personal keys is by design very limited:
# The rate limit for a personal keys is by design very limited:
# 20 requests every 1 second
# 100 requests every 2 minutes

import time
def request_limit_checker(request_num):
    if request_num != 0:
        print("request_limit_checker #ACTIVE")
        if request_num % 16 == 0: # 80% of 20
            print("sleep 1s")
            time.sleep(1)
            print("wake up")
        if request_num % 80 == 0: # 80% of 100
            print("sleep 120s")
            time.sleep(120)
            print("wake up")

In [16]:
#get game detail
import time
MATCH_V5 = "https://" + region + ".api.riotgames.com/lol/match/v5/matches/by-puuid/"
request_num = 0
start = 0;
count = 100;
number_of_request = 10; #get 10000 games
game_name_arr = []
classic_game_history_csv_path = "../data/"+ summoner_name_for_save_path +"game_history/classic_game_history.csv"
classic_game_name_history_json_path = "../data/"+ summoner_name_for_save_path +"game_history/classic_game_name_history.json"

for per_request in range(number_of_request): 
    request_num += 1
    request_limit_checker(request_num)
    MATCH_V5_URL = MATCH_V5 + encryptedPUUID + "/ids?start=" + str(start) + "&count=" + str(count) + "&api_key=" + api_key
    response = requests.get(MATCH_V5_URL)
    game_names_per_request = response.json()
    print("#PROGRESSING: Iteration: " + str(start))
    if len(game_names_per_request) == 0:
        print("#FINISH: No more games")
        break
    else:
        for game_name in game_names_per_request:
            print("#PROGRESSING: Iteration: " + str(start) + "." + str(request_num))
            request_num+=1
            request_limit_checker(request_num)
            game_detail = get_game_detail(game_name) #json format to string(first data)
            #save game detail to json file
            json_response = json.dumps(game_detail)
            json_file_path = "../data/" + "log" + ".json"
            with open(json_file_path, 'w') as outfile:
                outfile.write(json_response)

            gameType = game_detail['info']['gameType']
            gameMode = game_detail['info']['gameMode']
            if gameType == "MATCHED_GAME" and gameMode == "CLASSIC":
                game_name_arr += [game_name]
                summoner_and_opponent_info = get_summoner_and_opponent_info(game_detail)
                summoner_and_opponent_info_writer(classic_game_history_csv_path, summoner_and_opponent_info)
                # print("SUCCESS: " + str(iteration))
            # else:
            #     print("FAIL: " + str(iteration) + " " + gameType + " " + gameMode)
    start +=count #next 100 games when count = 100

save_game_name_arr_to_json(classic_game_name_history_json_path, game_name_arr)
print("#FINISH: save_game_name_arr_to_json")


request_limit_checker #ACTIVE
#PROGRESSING: Iteration: 0
#PROGRESSING: Iteration: 0.1
request_limit_checker #ACTIVE
#PROGRESSING: Iteration: 0.2
request_limit_checker #ACTIVE
#PROGRESSING: Iteration: 0.3
request_limit_checker #ACTIVE
#PROGRESSING: Iteration: 0.4
request_limit_checker #ACTIVE
#PROGRESSING: Iteration: 0.5
request_limit_checker #ACTIVE
#PROGRESSING: Iteration: 0.6
request_limit_checker #ACTIVE
#PROGRESSING: Iteration: 0.7
request_limit_checker #ACTIVE
#PROGRESSING: Iteration: 0.8
request_limit_checker #ACTIVE
#PROGRESSING: Iteration: 0.9
request_limit_checker #ACTIVE
#PROGRESSING: Iteration: 0.10
request_limit_checker #ACTIVE
#PROGRESSING: Iteration: 0.11
request_limit_checker #ACTIVE
#PROGRESSING: Iteration: 0.12
request_limit_checker #ACTIVE
#PROGRESSING: Iteration: 0.13
request_limit_checker #ACTIVE
#PROGRESSING: Iteration: 0.14
request_limit_checker #ACTIVE
#PROGRESSING: Iteration: 0.15
request_limit_checker #ACTIVE
sleep 1s
wake up
#PROGRESSING: Iteration: 0.16
reque